In [1]:
#IMPORTING THE LIBRARY FOR EDA AND READ EXCEL FILE
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [2]:
#TO IGNORE THE WARNING 
warnings.filterwarnings("ignore")

In [3]:
# GET THE DATA FROM EXCEL FILE WORKSHEETS
file_path = "Bird_Monitoring_Data_GRASSLAND.XLSX"
excel_data = pd.ExcelFile(file_path)
sheet_names = excel_data.sheet_names
sheets_dict = {sheet: excel_data.parse(sheet) for sheet in sheet_names}

In [4]:
#COMBINING THE SHEET FOR MAKING SINGLE DATAFRAME
combined_grassland_df = pd.concat(
    [df.assign(Sheet=sheet_name) for sheet_name, df in sheets_dict.items()],
    ignore_index=True
)

In [5]:
#DROP THE SHEET COLUMN NAME
combined_grassland_df = combined_grassland_df.drop(columns=['Sheet'])

In [6]:
#TO DISPLAY THE DATA WITH ALL COLUMNS
pd.set_option('display.max_columns',None)

In [ ]:
#DISPLAY DATAFRAME
combined_grassland_df.head()

In [ ]:
#CHECKING THE INFO ABOUT THE DATA FRAME
combined_grassland_df.info()

In [ ]:
#CHECKING THE COUNT OF ROW FOR DATAFRAME FOREST
combined_grassland_df.count()

In [ ]:
#CHECKING THE PROBABILITY FOR THE DATA
combined_grassland_df.describe()

In [ ]:
#CHECKING THE NULL DATA
combined_grassland_df.isnull().sum()

In [12]:
#DROPING THE SUB UNIT CODE COLUMN WHICH HAS ALL DATA EMPTY
combined_grassland_df.drop(columns=['Sub_Unit_Code'], inplace=True)

In [13]:
#DROPING THE SEX COLUMN WHICH HAS MANY DATA EMPTY
combined_grassland_df.drop(columns=['Sex'], inplace=True)

In [ ]:
#CHECK THE ID METHOD WHICH ROW HAS THE NULL VALUE AND STORE IT IN VARIABLE
combined_grassland_df[combined_grassland_df['ID_Method'].isna()]

In [ ]:
#FILTERING THE COMMON NAME RED-EYED-VIREO FOR FILLING ID METHOD NULL 
combined_grassland_df[combined_grassland_df['Common_Name'] == "American Crow"]

In [16]:
#FULL THE NULL WITH SINGING BECAUSE THE RED EYED VIREO BIRD ID METHOD IS SINGING
combined_grassland_df["ID_Method"].fillna("Calling",inplace=True)

In [ ]:
#CHECKING NULL VALUE FOR ACCEPTED TSN
combined_grassland_df[combined_grassland_df['AcceptedTSN'].isna()]

In [ ]:
#CHECK THE AOU CODE WHICH IS COMMON IN ALL EMPTY ACCEPTED TSN
combined_grassland_df[combined_grassland_df['AOU_Code'] == "HOFI"]

In [19]:
#AFTER CHECKING THE COMMON COLUMN FOR ACCEPTED TSN ALL ARE EMPTY SO FILL NULL BY NULL
combined_grassland_df["AcceptedTSN"].fillna(0,inplace=True)

In [20]:
#FILLING THE NULL DATA BY MODE
mode_value = combined_grassland_df['Distance'].mode()[0]
combined_grassland_df['Distance'].fillna(mode_value, inplace=True)

In [ ]:
#CHECKING NULL VALUE FOR TAXONCODE
combined_grassland_df[combined_grassland_df['TaxonCode'].isna()]

In [ ]:
#CHECK THE AOU CODE WHICH IS COMMON IN ALL EMPTY ACCEPTED TAXONCODE
combined_grassland_df[combined_grassland_df["AOU_Code"]=='NOCR']

In [ ]:
#CHECK THE AOU CODE WHICH IS COMMON IN ALL EMPTY ACCEPTED TAXONCODE
combined_grassland_df[combined_grassland_df["AOU_Code"]=='CHPH']

In [24]:
#FILLING THE NULL VALUE WITH 0 IN TAXON CODE
combined_grassland_df["TaxonCode"].fillna('0',inplace=True)

In [ ]:
#AGAIN CHECKING THE NULL VALUE AFTER FILLING THE NULL
combined_grassland_df.isna().sum()

In [26]:
#ROUND THE TEMPERATURE AND HUMIDITY DATA FOR ANALYSIS
combined_grassland_df["Temperature"]=combined_grassland_df["Temperature"].round(0).astype(int)
combined_grassland_df["Humidity"]=combined_grassland_df["Humidity"].round(0).astype(int)

In [ ]:
#CHECK THE UNIQUE DATA IN SKY FOR REDUCE THE UNIQUE COLUMN
combined_grassland_df["Sky"].unique()

In [28]:
#MAKING THE DF DATA LOOK GOOD AND EASY TO ANALYSE
combined_grassland_df["Sky"].replace("Clear or Few Clouds","Clear/Few Clouds",inplace=True)

In [ ]:
#CHECKING THE SKY COLUMN AFTER REPLACE THE DATA
combined_grassland_df["Sky"].unique()

In [30]:
#SEPEARTE THE DAY AND MONTH IN NEW COLUMNS FOR EDA 
combined_grassland_df['Day'] = combined_grassland_df['Date'].dt.day
combined_grassland_df['Month'] = combined_grassland_df['Date'].dt.month

In [ ]:
#DISPLAY THE DATAFRAME AFTER CLEANING 
combined_grassland_df.head()

In [ ]:
#CHECKING NULL VALUES AFTER CLEANING THE DATA
combined_grassland_df.isna().sum()

In [88]:
combined_grassland_df.to_csv("CLEANED_GRASSLAND_BIRDS_MONITORING_DATA.csv",index = False)

EDA UNI-VARIENT ANALYSIS

In [ ]:
#DISPLAY THE BIRD OBSERVED COUNT BY OBSERVER
plt.figure(figsize=(6,6))
sns.countplot(x="Observer",data=combined_grassland_df,palette='viridis') 
plt.title("BIRD OBSERVED COUNT BY OBSERVER")
plt.xlabel("OBSERVER NAME")
plt.ylabel("BIRD OBSERVED COUNT")
plt.show()

In [ ]:

#DISPLAY THE COUNT OF RARE SPECIES
plt.figure(figsize=(5, 6))
sns.countplot(x="PIF_Watchlist_Status",data=combined_grassland_df,palette='bright') 
plt.title(" COUNT OF RARE SPECIES")
plt.xlabel("RARE SPECIES STATUS")
plt.ylabel("COUNT OF RARE SPECIES")
plt.show()

In [ ]:

#DISPLAY THE COUNT OF BIRDS ID METHOD AND ANALYSED MOST BIRD VISITED ARE SINGING BIRDS
plt.figure(figsize=(5, 6))
sns.countplot(x="ID_Method",data=combined_grassland_df,palette='viridis') 
plt.title(" COUNT OF BIRD ID METHOD")
plt.xlabel("BIRDS ID METHOD")
plt.ylabel("COUNT OF ID METHOD")
plt.show()

In [ ]:
#DISPLAY THE COUNT OF BIRD VISITED IN FIRST THREE MINUTES
plt.figure(figsize=(5,4))
sns.countplot(x = "Initial_Three_Min_Cnt",data =combined_grassland_df,palette='bright')
plt.title(" COUNT OF BIRD VISITED IN FIRST THREE MINUTES")
plt.xlabel("Initial_Three_Min_Cnt")
plt.ylabel("COUNT OF FIRST THREE MINUTES VISITED BIRDS")
plt.show()

EDA BI-VARIENT ANALYSIS

In [ ]:
#DISPLAY THE TOP VISIT COUNT OF BIRDS BY REGION
bird_counts = combined_grassland_df.groupby(["Admin_Unit_Code", "Common_Name"]).size().reset_index(name="count")
most_visited = bird_counts.loc[bird_counts.groupby("Admin_Unit_Code")["count"].idxmax()]
plt.figure(figsize=(10, 6))
sns.barplot(data=most_visited, x="Admin_Unit_Code", y="count", hue="Common_Name")
plt.title("Most Visited Bird per Region")
plt.xlabel("Region (anti_unit_code)")
plt.ylabel("Visit Count of bird")
plt.legend(title="Bird Name")
plt.show()

In [ ]:
#Categorical Data: Use deep, bright, or muted.
#Ordered Data: Use sequential palettes like Blues or viridis.
#Diverging Data: Use coolwarm or RdBu for two extremes.
#DISPLAY THE COUNT OF PLOT COUNT BY REGION
unique_sites = combined_grassland_df.groupby("Admin_Unit_Code")["Plot_Name"].nunique().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(data=unique_sites, x="Admin_Unit_Code", y="Plot_Name",palette="viridis")                   
plt.title("UNIQUE PLOT COUNT PER REGION")
plt.xlabel("Region(Admin_Unit_Code)")
plt.ylabel("Unique Plot Count")
plt.xticks(rotation=45)
plt.show()

In [ ]:
#DISPLAY THE MOST FREQUENT BIRD VISITS FOR EACH TEMPERATURE
bird_counts = combined_grassland_df.groupby(["Temperature", "Common_Name"]).size().reset_index(name="Count")
most_visited_bird = bird_counts.loc[bird_counts.groupby("Temperature")["Count"].idxmax()]
plt.figure(figsize=(17,7))
sns.barplot(data=most_visited_bird, x="Temperature", y="Count", hue="Common_Name", palette="viridis")
plt.title("MOST FREQUENT BIRD VISITS FOR EACH TEMPERATURE")
plt.xlabel("Temperature range")
plt.ylabel("Count of most visited bird for each temp")
plt.show()

In [ ]:
#DISPLAY THE MOSTED VISITED BIRD BY SKY 
bird_count_by_sky = combined_grassland_df.groupby(["Sky", "Common_Name"]).size().reset_index(name="Count")
bird_count_by_sky_max = bird_count_by_sky.loc[bird_count_by_sky.groupby("Sky")["Count"].idxmax()]
sns.barplot(data=bird_count_by_sky_max , x="Sky", y="Count", hue="Common_Name", palette="viridis")
plt.title("MOST FREQUENT BIRD VISITS BY SKY")
plt.xlabel("Sky type")
plt.ylabel("Count of birds")
plt.xticks(rotation=45)
plt.show()

In [ ]:
#CHECKING THE RELATION BETWEEN TEMP AND HUMIDITY
correlation = combined_grassland_df['Temperature'].corr(combined_grassland_df['Humidity'])
correlation

In [ ]:
#DISPLAY THE MOST FREQUENT BIRD VISITS FOR EACH HUMIDITY
bird_counts = combined_grassland_df.groupby(["Humidity", "Common_Name"]).size().reset_index(name="Count")
most_visited_bird = bird_counts.loc[bird_counts.groupby("Humidity")["Count"].idxmax()]
plt.figure(figsize=(15,8))
sns.barplot(data=most_visited_bird, x="Humidity", y="Count", hue="Common_Name")
plt.title("MOST FREQUENT BIRD VISITS FOR EACH HUMIDITY")
plt.xlabel("Humidity level")
plt.ylabel("Frequent bird visit count")
plt.xticks(rotation=90)
plt.show()

EDA MULTI-VARIENT ANALYSIS

In [ ]:
#CHECKING THE RELATION BETWEEN ALL THE NUMERIC DATA USING PAIR PLOT
numeric_columns = ['Month', 'Visit', 'AcceptedTSN','Temperature','Humidity']
filtered_data = combined_grassland_df[numeric_columns]
sns.pairplot(filtered_data)
sns.set_palette("coolwarm")
plt.show()